In [ ]:
!mkdir -p input
!wget -q https://github.com/several27/FakeNewsCorpus/releases/download/v1.0/news.csv.z01 -O input/news.csv.z01
!wget -q https://github.com/several27/FakeNewsCorpus/releases/download/v1.0/news.csv.z02 -O input/news.csv.z02
!wget -q https://github.com/several27/FakeNewsCorpus/releases/download/v1.0/news.csv.z03 -O input/news.csv.z03
!wget -q https://github.com/several27/FakeNewsCorpus/releases/download/v1.0/news.csv.z04 -O input/news.csv.z04
!wget -q https://github.com/several27/FakeNewsCorpus/releases/download/v1.0/news.csv.z05 -O input/news.csv.z05
!wget -q https://github.com/several27/FakeNewsCorpus/releases/download/v1.0/news.csv.z06 -O input/news.csv.z06
!wget -q https://github.com/several27/FakeNewsCorpus/releases/download/v1.0/news.csv.z07 -O input/news.csv.z07
!wget -q https://github.com/several27/FakeNewsCorpus/releases/download/v1.0/news.csv.z08 -O input/news.csv.z08
!wget -q https://github.com/several27/FakeNewsCorpus/releases/download/v1.0/news.csv.z09 -O input/news.csv.z09
!wget -q https://github.com/several27/FakeNewsCorpus/releases/download/v1.0/news.csv.zip -O input/news.csv.zip

In [ ]:
!cat input/news.csv.z* > input/all.zip

In [ ]:
!unzip input/all.zip -d input/

Archive:  input/all.zip
error: End-of-centdir-64 signature not where expected (prepended bytes?)
  (attempting to process anyway)
warning [input/all.zip]:  zipfile claims to be last disk of a multi-part archive;
  attempting to process anyway, assuming all parts have been concatenated
  together in order.  Expect "errors" and warnings...true multi-part support
  doesn't exist yet (coming soon).
warning [input/all.zip]:  9663676416 extra bytes at beginning or within zipfile
  (attempting to process anyway)
file #1:  bad zipfile offset (local header sig):  9663676420
  (attempting to re-compensate)
  inflating: input/news_cleaned_2018_02_13.csv  


In [90]:
import tensorflow as tf
import itertools
import random
import csv

In [ ]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [ ]:
def serialize_example(text):
  feature = {
      "text": _bytes_feature(text.encode("utf-8")),
  }

  example = tf.train.Example(features=tf.train.Features(feature=feature))
  return example.SerializeToString()

In [63]:
def grouper(iterable, n):
    while True:
        yield itertools.chain((next(iterable),), itertools.islice(iterable, n-1))

In [70]:
project_id = "alaska2-280023"
bucket_name = "dbunk"

In [71]:
from google.colab import auth
auth.authenticate_user()

In [76]:
!gcloud config set project {project_id}

Updated property [core/project].


In [81]:
with open("input/news_cleaned_2018_02_13.csv") as csvfile:
    i = 0
    reader = csv.DictReader(csvfile)
    reader = (row for row in reader if row["content"])
    for chunk in grouper(reader, 65536):
        if i >= 101:
            print(f"Writing {i}.tfrecord")
            with tf.io.TFRecordWriter(f"{i}.tfrecord") as writer:
                for row in chunk:
                    if row["title"]:
                        text = row["title"] + "\n\n" + row["content"]
                    else:
                        text = row["content"]
                    example = serialize_example(text)
                    writer.write(example)
            !gsutil cp {i}.tfrecord gs://{bucket_name}/
            !rm {i}.tfrecord
        else:
            print(f"Skipping {i}.tfrecord")

        i += 1

Skipping 0.tfrecord
Skipping 1.tfrecord
Skipping 2.tfrecord
Skipping 3.tfrecord
Skipping 4.tfrecord
Skipping 5.tfrecord
Skipping 6.tfrecord
Skipping 7.tfrecord
Skipping 8.tfrecord
Skipping 9.tfrecord
Skipping 10.tfrecord
Skipping 11.tfrecord
Skipping 12.tfrecord
Skipping 13.tfrecord
Skipping 14.tfrecord
Skipping 15.tfrecord
Skipping 16.tfrecord
Skipping 17.tfrecord
Skipping 18.tfrecord
Skipping 19.tfrecord
Skipping 20.tfrecord
Skipping 21.tfrecord
Skipping 22.tfrecord
Skipping 23.tfrecord
Skipping 24.tfrecord
Skipping 25.tfrecord
Skipping 26.tfrecord
Skipping 27.tfrecord
Skipping 28.tfrecord
Skipping 29.tfrecord
Skipping 30.tfrecord
Skipping 31.tfrecord
Skipping 32.tfrecord
Skipping 33.tfrecord
Skipping 34.tfrecord
Skipping 35.tfrecord
Skipping 36.tfrecord
Skipping 37.tfrecord
Skipping 38.tfrecord
Skipping 39.tfrecord
Skipping 40.tfrecord
Skipping 41.tfrecord
Skipping 42.tfrecord
Skipping 43.tfrecord
Skipping 44.tfrecord
Skipping 45.tfrecord
Skipping 46.tfrecord
Skipping 47.tfrecord
Sk

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: generator 'grouper' raised StopIteration
  """


In [92]:
GCS_PATH = "gs://dbunk"
filenames = tf.io.gfile.glob(GCS_PATH + "/*.tfrecord")
random.shuffle(filenames)

In [95]:
feature_description = {
    "text": tf.io.FixedLenFeature([], tf.string),
}

def parse(example):
    return tf.io.parse_single_example(example, feature_description)["text"]

dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=tf.data.experimental.AUTOTUNE)
dataset = dataset.map(parse)
dataset = dataset.shuffle(10000)

In [104]:
for record in dataset.take(1):
    print(record.numpy().decode("utf-8"))

Obama’s Latest Whopper—-Let’s Raise Social Security Benefits! – InvestmentWatch

The U.S. has approximately $$80 trillion of unfunded liabilities for social security, medicare and other entitlements sitting atop a work force that is rapidly aging and an economy that is lapsing into stasis. Yet in the midst of a campaign diatribe about Donald Trump’s alleged lack of preparation for the highest office in the land, the current White House occupant proved that in nearly eight years he has learned exactly nothing about the nation’s abysmal fiscal plight.

“And not only do we need to strengthen its long-term health, it’s time we finally made Social Security more generous and increased its benefits so that today’s retirees and future generations get the dignified retirement that they’ve earned,” Obama said in an economic call to arms in Elkhart, Indiana.

Don’t bother to say he must be kidding. After all, our President also claims the disaster known as Obamacare is a roaring success; and that